In [2]:
# !pip install -r requirements.txt

In [2]:
!mkdir feature_store/data
!cp ../../session3/data/house_dataset_main.parquet feature_store/data/
!cp ../../session3/data/house_dataset_lat_lon.parquet feature_store/data/

In [3]:
import pandas as pd
pd.read_parquet('feature_store/data/house_dataset_main.parquet').head(2)

,HouseId,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,EventTimestamp,Created
0,1,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,2021-12-11 18:40:03,2022-04-12 12:04:13
1,2,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,2021-12-11 18:57:30,2022-04-12 12:04:13


In [4]:
pd.read_parquet('feature_store/data/house_dataset_lat_lon.parquet').head(2)

,HouseId,Latitude,Longitude,EventTimestamp,Created
0,1,34.18,-118.38,2021-12-11 18:40:03,2022-02-13 22:43:53
1,2,39.08,-121.04,2021-12-11 18:57:30,2022-02-13 22:43:53


In [5]:
cd feature_store

/home/ubuntu/dvoitekh/aihouse_mlops_course/session4/feast/feature_store


In [6]:
!feast --help

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Usage: feast [OPTIONS] COMMAND [ARGS]...

  Feast CLI

  For more information, see our public docs at https://docs.feast.dev/

  For any questions, you can reach us at https://slack.feast.dev/

Options:
  -c, --chdir TEXT           Switch to a different feature repository
                             directory before executing the given subcommand.
  --log-level TEXT           The logging level. One of DEBUG, INFO, WARNING,
                             ERROR, and CRITICAL (case-insensitive).
  --feature-store-yaml TEXT  Override the directory where the CLI should look
                             for the feature_store.yaml file.
  --help                     Show this message and exit.

Commands:
  apply                    Create or 

In [7]:
!feast plan

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Created entity HouseId
Created feature view house_lat_lon_view
Created feature view house_main_view
Created feature service house_service

No changes to infrastructure


In [8]:
!feast apply

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Created entity HouseId
Created feature view house_main_view
Created feature view house_lat_lon_view
Created feature service house_service

Deploying infrastructure for house_main_view
Deploying infrastructure for house_lat_lon_view


In [9]:
import os
from feast import FeatureStore

store = FeatureStore(repo_path=".")

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [10]:
import feast
import pandas as pd
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

entity_df = pd.DataFrame.from_dict({"HouseId": [i for i in range(1, 1000)]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

retrieval_job = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
)

In [11]:
retrieval_job.metadata.features

['house_main_view:MedInc',
 'house_main_view:HouseAge',
 'house_main_view:AveRooms',
 'house_main_view:AveBedrms',
 'house_main_view:Population',
 'house_main_view:AveOccup',
 'house_main_view:MedHouseVal',
 'house_lat_lon_view:Latitude',
 'house_lat_lon_view:Longitude']

In [12]:
dataset = store.create_saved_dataset(
    from_=retrieval_job,
    name='merged_dataset_v1',
    storage=SavedDatasetFileStorage('merged_dataset_v1.parquet')
)

training_df = retrieval_job.to_df()
print(len(training_df))
training_df.head()

/home/ubuntu/.local/lib/python3.9/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


999


,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-10-26 21:25:57.755422+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,34.18,-118.38
1,2,2022-10-26 21:25:57.755422+00:00,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,39.08,-121.04
2,3,2022-10-26 21:25:57.755422+00:00,3.7500,16.0,5.768719,1.023295,1478.0,2.459235,1.473,38.68,-121.28
3,4,2022-10-26 21:25:57.755422+00:00,2.8542,34.0,3.858779,1.045802,1164.0,4.442748,1.469,34.04,-118.19
4,5,2022-10-26 21:25:57.755422+00:00,1.3375,18.0,4.567625,1.087327,2707.0,2.882854,0.596,39.13,-121.54


In [13]:
!feast materialize-incremental 2022-02-01T00:00:00

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Materializing 2 feature views to 2022-02-01 00:00:00+00:00 into the redis online store.

Since the ttl is 0 for feature view house_main_view, the start date will be set to 1 year before the current time.
house_main_view from 2021-10-27 21:26:04+00:00 to 2022-02-01 00:00:00+00:00:
100%|█████████████████████████

In [14]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,HouseAge,MedHouseVal,MedInc,Latitude,Longitude
0,1,24.0,2.000,2.4792,34.180000,-118.379997
1,20640,13.0,3.333,3.3438,33.689999,-117.800003


In [15]:
!feast materialize-incremental 2022-06-11T00:00:00

/home/ubuntu/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ubuntu/.local/lib/python3.9/site-packages/feast/repo_config.py:216: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(
Materializing 2 feature views to 2022-06-11 00:00:00+00:00 into the redis online store.

house_main_view from 2022-02-01 00:00:00+00:00 to 2022-06-11 00:00:00+00:00:
100%|██████████████████████████████████████████████████████| 14819/14819 [00:00<00:00, 25447.79it/s]
house_lat_lon_view from 2022-02-01 00:00:00+

In [16]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,HouseAge,MedHouseVal,MedInc,Latitude,Longitude
0,1,24.0,2.000,2.4792,34.180000,-118.379997
1,20640,13.0,3.333,3.3438,33.689999,-117.800003


In [17]:
import datetime

store.push('main_push_source', pd.DataFrame([{
    'HouseId': 1,
    'MedInc': 1.0,
    'HouseAge': 1.0,
    'AveRooms': 1.0,
    'AveBedrms': 1.0,
    'Population': 1.0,
    'AveOccup': 1.0,
    'MedHouseVal': 1.0,
    'Latitude': 1.0,
    'Longitude': 1.0,
    'EventTimestamp': datetime.datetime.now(),
    'Created': datetime.datetime.now(),
}]))

In [18]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,HouseAge,MedHouseVal,MedInc,Latitude,Longitude
0,1,1.0,1.000,1.0000,34.180000,-118.379997
1,20640,13.0,3.333,3.3438,33.689999,-117.800003


In [19]:
entity_df = pd.DataFrame.from_dict({"HouseId": [1]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
).to_df()

,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-10-26 21:26:36.184565+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.0,34.18,-118.38


In [20]:
!redis-cli INFO Keyspace

# Keyspace
db0:keys=20641,expires=0,avg_ttl=0


Also, we can run Feast UI to track features and datasets